In [19]:
import numpy as np
import math
import glob
import os
import umap
import hdbscan
import seaborn as sns
import pandas as pd
import pickle
import openpyxl
import cv2
from collections import defaultdict

In [77]:
pilot3_rewardcomp = pd.read_excel(r"C:\Users\megha\Documents\Padilla-Coreano\DomComp\rewardcomp\iwata_pilot_3_reward_competition.xlsx")
pilot2_rewardcomp = pd.read_excel(r"C:\Users\megha\Documents\Padilla-Coreano\DomComp\rewardcomp\iwata_pilot_2_reward_competition.xlsx")
pilot1_rewardcomp = pd.read_excel(r"C:\Users\megha\Documents\Padilla-Coreano\DomComp\rewardcomp\cum_pilot_1_reward_competition.xlsx")


In [78]:
def extract_mouse1(match):
    return(extract_mice_identity(match)[0])
def extract_mouse2(match):
    return(extract_mice_identity(match)[1])

def extract_mice_identity(match):
    pieces = match.split(' ')
    if len(pieces) == 3:
        mouse1 = pieces[0]
        mouse2 = pieces[2]
    else:
        pieces = match.split("vs")
        if len(pieces) > 1:
            mouse1 = pieces[0]
            mouse2 = pieces[1]
        else:
            pieces = match.split("v")
            mouse1 = pieces[0]
            mouse2 = pieces[1]
    return(mouse1, mouse2)




In [80]:
def reward_comp_matrices(rewardcomp_df):
    matches = rewardcomp_df['Match'].values
    mouse1wins = rewardcomp_df['Mouse 1 Wins'].values
    mouse2wins = rewardcomp_df['Mouse 2 Wins'].values
    matrices = create_matrices(matches)
    full_matrices = fill_matrices(matches, mouse1wins, mouse2wins, matrices)
    return(full_matrices)

def create_matrices(matches):
    subjects = []
    for match in matches:
        pieces = match.split(' ')
        if len(pieces) == 3:
            subjects.append(pieces[0])
            subjects.append(pieces[2])
        else:
            pieces = match.split("vs")
            if len(pieces) > 1:
                subjects.append(pieces[0])
                subjects.append(pieces[1])
            else:
                pieces = match.split("v")
                subjects.append(pieces[0])
                subjects.append(pieces[1])
                
    subjects = np.unique(subjects)
    cages = defaultdict(list)
    for subject in subjects: 
        cages[subject[0]].append(subject)

    matrices = defaultdict()
    for cage, subjects in cages.items():
        matrices[cage] = pd.DataFrame(index = subjects, columns = subjects, data = 0)
    return(matrices)

def fill_matrices(matches, mouse1wins, mouse2wins, matrices):
    for i in range(len(matches)):
        pieces = matches[i].split(' ')
        if len(pieces) == 3:
            mouse1 = pieces[0]
            mouse2 = pieces[2]
        else:
            pieces = matches[i].split("vs")
            if len(pieces) > 1:
                mouse1 = pieces[0]
                mouse2 = pieces[1]
            else:
                pieces = matches[i].split("v")
                mouse1 = pieces[0]
                mouse2 = pieces[1]
        mouse1win = mouse1wins[i]
        mouse2win = mouse2wins[i]
        cage = mouse1[0]
        matrices[cage].loc[mouse1,mouse2] = mouse1win + matrices[cage].loc[mouse1,mouse2]
        matrices[cage].loc[mouse2,mouse1] = mouse2win + matrices[cage].loc[mouse2,mouse1]
    return(matrices)

 
pilot3_matrices = reward_comp_matrices(pilot3_rewardcomp)
pilot2_matrices = reward_comp_matrices(pilot2_rewardcomp)
pilot1_matrices = reward_comp_matrices(pilot1_rewardcomp)




,4.1,4.2,4.3,4.4
4.1,0,34,20,20
4.2,5,0,13,7
4.3,14,20,0,17
4.4,18,30,21,0
